In [15]:
import cv2
import numpy as np
import os
import pandas as pd

# Convert video to a specific FPS
def convert_to_fps(input_path, output_path, fps):
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print(f"Error: Could not open the video {input_path}.")
        return
    
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        out.write(frame)

    cap.release()
    out.release()

# Apply Gaussian blur to reduce noise
def apply_noise_reduction(frame):
    return cv2.GaussianBlur(frame, (5, 5), 0)

# Analyze a single video and extract features
def analyze_video(video_path, target_fps):
    # Convert video to target FPS
    temp_output_video_path = 'temp_video_30fps.mp4'
    convert_to_fps(video_path, temp_output_video_path, target_fps)

    cap = cv2.VideoCapture(temp_output_video_path)
    if not cap.isOpened():
        print(f"Error: Could not open the video {temp_output_video_path}.")
        return None

    # Variables for gait analysis
    step_lengths = []
    walking_speeds = []
    frame_times = []

    prev_frame = None
    prev_contour_centroid = None

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        current_time = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0
        frame_times.append(current_time)

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = apply_noise_reduction(gray)

        if prev_frame is not None:
            frame_diff = cv2.absdiff(prev_frame, gray)
            _, thresh = cv2.threshold(frame_diff, 30, 255, cv2.THRESH_BINARY)
            contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            for contour in contours:
                area = cv2.contourArea(contour)
                if area > 1000:
                    M = cv2.moments(contour)
                    if M["m00"] != 0:
                        cX = int(M["m10"] / M["m00"])
                        cY = int(M["m01"] / M["m00"])
                        contour_centroid = (cX, cY)

                        if prev_contour_centroid is not None:
                            step_length = np.linalg.norm(np.array(contour_centroid) - np.array(prev_contour_centroid))
                            step_lengths.append(step_length)

                            walking_speed = step_length * target_fps
                            walking_speeds.append(walking_speed)

                        prev_contour_centroid = contour_centroid

                    x, y, w, h = cv2.boundingRect(contour)
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        prev_frame = gray.copy()
        cv2.imshow('Gait Analysis', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    avg_step_length = np.mean(step_lengths) if step_lengths else 0
    avg_stride_length = avg_step_length * 2
    avg_walking_speed = np.mean(walking_speeds) if walking_speeds else 0

    if len(walking_speeds) > 1 and len(frame_times) > 1:
        frame_times_diff = np.diff(frame_times)
        walking_speeds_diff = np.diff(walking_speeds)
        min_length = min(len(frame_times_diff), len(walking_speeds_diff))
        frame_times_diff = frame_times_diff[:min_length]
        walking_speeds_diff = walking_speeds_diff[:min_length]
        frame_times_diff[frame_times_diff == 0] = np.nan
        accelerations = walking_speeds_diff / frame_times_diff
        avg_acceleration = np.nanmean(accelerations)
    else:
        avg_acceleration = np.nan

    variance_step_length = np.var(step_lengths) if step_lengths else 0

    cap.release()
    cv2.destroyAllWindows()

    return {
        "video": os.path.basename(video_path),
        "avg_step_length": avg_step_length,
        "avg_stride_length": avg_stride_length,
        "avg_walking_speed": avg_walking_speed,
        "avg_acceleration": avg_acceleration,
        "variance_step_length": variance_step_length
    }

# Absolute path for the input folder
input_folder_path = 'C:\\Users\\Rajarshi Das\\Repo Behavioral Pattern Analysis\\Gait Features Extraction\\Input'
target_fps = 30

# List to store the results
results = []

# Loop through all the videos in the folder
for video_file in os.listdir(input_folder_path):
    if video_file.endswith('.mp4'):
        video_path = os.path.join(input_folder_path, video_file)
        features = analyze_video(video_path, target_fps)
        if features is not None:
            results.append(features)

# Create a DataFrame from the results
df = pd.DataFrame(results)

# Display the DataFrame
print(df)

# Save the DataFrame to a CSV file
df.to_csv('gait_analysis_results.csv', index=False)


     video  avg_step_length  avg_stride_length  avg_walking_speed  \
0    1.mp4       437.231731         874.463461       13116.951916   
1   11.mp4       473.389427         946.778853       14201.682795   
2   12.mp4       445.967162         891.934325       13379.014873   
3   13.mp4       278.139096         556.278192        8344.172887   
4   14.mp4       488.771776         977.543551       14663.153270   
5    2.mp4       308.684184         617.368367        9260.525511   
6    3.mp4       441.472776         882.945553       13244.183291   
7    4.mp4       599.980420        1199.960841       17999.412612   
8    5.mp4       410.024848         820.049697       12300.745453   
9    6.mp4       434.618627         869.237255       13038.558819   
10   7.mp4       457.755750         915.511500       13732.672503   
11   9.mp4       501.826625        1003.653250       15054.798754   

    avg_acceleration  variance_step_length  
0        3591.913856         200591.460016  
1           

Methods used for feature selection and feature extraction

### ALGORITHM

#### Function to convert video to a specific FPS
def convert_to_fps(input_path, output_path, fps):
    Initialize video capture from input_path
    If video capture is not opened:
        Print error message
        Return

    Get frame width and height
    Initialize video writer with output_path, codec, fps, and frame size

    While video capture is successful:
        Read a frame
        If frame read is unsuccessful:
            Break loop
        Write frame to output video

    Release video capture and writer

#### Function to apply Gaussian blur for noise reduction
def apply_noise_reduction(frame):
    Apply Gaussian blur with 5x5 kernel to the frame
    Return blurred frame

#### Function to analyze a single video and extract features
def analyze_video(video_path, target_fps):
    Convert video to target FPS and save as temp_output_video_path
    Initialize video capture from temp_output_video_path
    If video capture is not opened:
        Print error message
        Return None

    Initialize lists for step_lengths, walking_speeds, and frame_times
    Initialize prev_frame and prev_contour_centroid to None

    While video capture is successful:
        Read a frame
        If frame read is unsuccessful:
            Break loop

        Get current frame timestamp in seconds
        Append timestamp to frame_times

        Convert frame to grayscale
        Apply noise reduction to grayscale frame

        If prev_frame is not None:
            Compute absolute difference between prev_frame and current frame
            Apply binary thresholding to frame difference
            Find contours in thresholded image

            For each contour in contours:
                Compute contour area
                If area > 1000:
                    Compute contour's centroid
                    If prev_contour_centroid is not None:
                        Calculate step length as Euclidean distance between current and previous centroids
                        Append step length to step_lengths
                        Calculate walking speed as step length times target_fps
                        Append walking speed to walking_speeds
                    Update prev_contour_centroid with current centroid
                    Draw bounding box around contour

        Update prev_frame with current frame
        Display current frame
        If 'q' is pressed:
            Break loop

    Calculate avg_step_length, avg_stride_length, avg_walking_speed
    If sufficient data is available, calculate avg_acceleration
    Calculate variance_step_length

    Release video capture and close all OpenCV windows

    Return dictionary with extracted features

#### Main script
Set input_folder_path to the absolute path of the input folder
Set target_fps to 30

Initialize results list

For each video_file in input_folder_path:
    If video_file is an MP4 file:
        Get video_path by joining input_folder_path and video_file
        Extract features using analyze_video
        If features are not None:
            Append features to results

Create DataFrame from results list
Print DataFrame
Save DataFrame to CSV file 'gait_analysis_results.csv'


### METHODOLOGY

#### Introduction

This methodology outlines the steps taken to analyze gait features from video data. The process involves converting videos to a specified frame rate, reducing noise using Gaussian blur, and extracting features such as step length, stride length, walking speed, and acceleration. The steps are implemented in Python using OpenCV and numpy libraries. The extracted features are saved in a CSV file for further analysis.

#### 1. Video Frame Rate Conversion

**Objective:** Convert the video to a specified frame rate (FPS) to standardize the frame rate for analysis.

**Method:**

1. **Initialization:**
   - Open the input video using `cv2.VideoCapture(input_path)`.
   - Check if the video is successfully opened. If not, print an error message and return.
   
2. **Retrieve Frame Dimensions:**
   - Get the width and height of the frames using `cv2.CAP_PROP_FRAME_WIDTH` and `cv2.CAP_PROP_FRAME_HEIGHT`.

3. **Initialize Video Writer:**
   - Use `cv2.VideoWriter` to create a writer object with the specified `fps`, frame size, and codec (`mp4v`).

4. **Frame-by-Frame Conversion:**
   - Read frames from the input video.
   - Write each frame to the output video.
   - Release the video capture and writer resources.

**Mathematical Expression:**
\[ \text{New Video FPS} = \text{target\_fps} \]

#### 2. Noise Reduction Using Gaussian Blur

**Objective:** Reduce noise in the video frames to enhance contour detection.

**Method:**

1. **Gaussian Blur Application:**
   - Apply Gaussian blur to each frame using `cv2.GaussianBlur(frame, (5, 5), 0)`.
   - This step smoothens the image, reducing high-frequency noise.

**Mathematical Expression:**
\[ \text{Gaussian Blur} = \frac{1}{2\pi\sigma^2} e^{-\frac{x^2 + y^2}{2\sigma^2}} \]

#### 3. Gait Feature Extraction

**Objective:** Extract gait features such as step length, stride length, walking speed, and acceleration from the video.

**Method:**

1. **Initialize Variables:**
   - Initialize lists for `step_lengths`, `walking_speeds`, and `frame_times`.
   - Initialize `prev_frame` and `prev_contour_centroid` to `None`.

2. **Frame Processing:**
   - Convert each frame to grayscale using `cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)`.
   - Apply Gaussian blur for noise reduction.
   
3. **Frame Difference Calculation:**
   - Compute the absolute difference between the current frame and the previous frame using `cv2.absdiff(prev_frame, gray)`.
   - Apply binary thresholding to the frame difference to get a binary image highlighting the moving objects.

4. **Contour Detection:**
   - Detect contours in the thresholded image using `cv2.findContours`.
   - For each contour with an area greater than 1000 pixels:
     - Compute the centroid using image moments.
     - If a previous centroid exists, calculate the step length as the Euclidean distance between the current and previous centroids.
     - Compute the walking speed as the product of step length and target FPS.
     - Append the calculated step length and walking speed to their respective lists.

**Mathematical Expressions:**

- **Step Length:**
\[ \text{Step Length} = \sqrt{(x_2 - x_1)^2 + (y_2 - y_1)^2} \]

- **Walking Speed:**
\[ \text{Walking Speed} = \text{Step Length} \times \text{FPS} \]

- **Average Step Length:**
\[ \text{Average Step Length} = \frac{1}{N} \sum_{i=1}^{N} \text{Step Length}_i \]

- **Average Stride Length:**
\[ \text{Average Stride Length} = 2 \times \text{Average Step Length} \]

- **Average Walking Speed:**
\[ \text{Average Walking Speed} = \frac{1}{N} \sum_{i=1}^{N} \text{Walking Speed}_i \]

- **Average Acceleration:**
\[ \text{Acceleration} = \frac{\Delta \text{Speed}}{\Delta \text{Time}} \]
\[ \text{Average Acceleration} = \frac{1}{N-1} \sum_{i=1}^{N-1} \frac{\text{Speed}_{i+1} - \text{Speed}_i}{\text{Time}_{i+1} - \text{Time}_i} \]

- **Variance of Step Length:**
\[ \text{Variance of Step Length} = \frac{1}{N} \sum_{i=1}^{N} (\text{Step Length}_i - \text{Average Step Length})^2 \]

#### 4. Saving and Displaying Results

**Objective:** Save the extracted features in a CSV file and display the results.

**Method:**

1. **Collect Results:**
   - Create a list to store the results for each video.
   - Append the extracted features as a dictionary to the results list.

2. **Create DataFrame:**
   - Convert the results list into a pandas DataFrame.

3. **Save to CSV:**
   - Save the DataFrame to a CSV file named `gait_analysis_results.csv`.

4. **Display Results:**
   - Print the DataFrame to display the results.

#### Conclusion

This methodology provides a comprehensive approach to analyzing gait features from video data. The steps involve standardizing the frame rate, reducing noise, detecting contours, and extracting relevant features. The results are stored in a CSV file for further analysis. This approach can be used in various applications, including healthcare, sports, and surveillance.


In [3]:
df.head()

,video,avg_step_length,avg_stride_length,avg_walking_speed,avg_acceleration,variance_step_length
0,1.mp4,437.231731,874.463461,13116.951916,3591.913856,200591.460016
1,11.mp4,473.389427,946.778853,14201.682795,9.778961,233761.039858
2,12.mp4,445.967162,891.934325,13379.014873,104.504270,191248.910774
3,13.mp4,278.139096,556.278192,8344.172887,365.084698,145265.285443
4,14.mp4,488.771776,977.543551,14663.153270,-10078.321011,261127.709793
